In [ ]:
import pandas as pd
import numpy as np
from wmb import *
from ALLCools.plot import *

import pathlib

In [ ]:
mc_annot = cemba.get_mc_annot()
m3c_annot = cemba.get_m3c_annot()

In [ ]:
def save_index(cells, file_name, downsample=100000, random_state=0):
    cells = pd.Series(cells)
    if cells.size > downsample:
        cells = cells.sample(downsample, random_state=random_state)
    cells.to_csv(file_name, index=None, header=False)

## L1 - Neuron Non-Neuron

In [ ]:
# pathlib.Path('../L1/ALL').mkdir(parents=True, exist_ok=True)
# save_index(mc_annot.get_index('cell'),
#            file_name='../L1/ALL/mc_cells.txt',
#            downsample=3000000,
#            random_state=0)
# 
# save_index(m3c_annot.index,
#            file_name='../L1/ALL/m3c_cells.txt',
#            downsample=3000000,
#            random_state=0)

In [ ]:
# cwd = pathlib.Path().absolute()
# 
# with open('Snakefile') as f, open('../L1/Snakefile', 'w') as out_f:
#     snakefile_tmp = f.read()
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_TEMPLATE_DIR', str(cwd))
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_DATASET', 'CEMBA_3C')
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_CLUSTER_COL', str('L1'))
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_GROUP_KEY',
#                                           str('L2'))
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_PLOT_KEY',
#                                           str('L1'))
#     snakefile_tmp = snakefile_tmp.replace(
#         'REPLACE_CATEGORICAL_KEY', str(['L1_annot', 'L2', 'DissectionRegion']))
#     out_f.write(snakefile_tmp)

## L2

#### ALL

In [ ]:
mc_groups = pd.read_csv('../L1/ALL/mc_integration_group.csv.gz',
                        index_col=0).squeeze()
m3c_groups = pd.read_csv('../L1/ALL/m3c_integration_group.csv.gz',
                         index_col=0).squeeze()

In [ ]:
# note: L1 is run with 176147 cells, which accidentally contains some outliers, we updated the m3c_annot, the correct cell number should be 176003
# reindex m3c integration groups here

In [ ]:
m3c_groups = m3c_groups.loc[m3c_annot.get_index('cell')].copy()

In [ ]:
assert mc_groups.index.duplicated().sum() == 0
assert mc_groups.index.size == mc_annot.get_index('cell').size
assert mc_groups.isna().sum() == 0

assert m3c_groups.index.duplicated().sum() == 0
assert m3c_groups.index.size == m3c_annot.get_index('cell').size
assert m3c_groups.isna().sum() == 0

In [ ]:
m3c_groups[m3c_groups.isin([40, 39, 0])]

In [ ]:
for integration_group in m3c_groups.unique():
    mc_cells = mc_groups[mc_groups  == integration_group].index
    m3c_cells = m3c_groups[m3c_groups == integration_group].index

    integration_group = f'InteGroup{integration_group}'
    
    if mc_cells.size <= 150 or m3c_cells.size <= 150:
        print(integration_group)

In [ ]:
# # for L2 ALL
# for integration_group in m3c_groups.unique():
#     mc_cells = mc_groups[mc_groups  == integration_group].index
#     m3c_cells = m3c_groups[m3c_groups == integration_group].index
# 
#     integration_group = f'InteGroup{integration_group}'
#     
#     
#     if mc_cells.size <= 150 or m3c_cells.size <= 150:
#         print(integration_group)
#     else:
#         out_dir = pathlib.Path(f'../L2/ALL/{integration_group}')
#         out_dir.mkdir(exist_ok=True, parents=True)
#         save_index(mc_cells,
#                    f'../L2/ALL/{integration_group}/mc_cells.txt',
#                    downsample=1000000,
#                    random_state=0)
#         save_index(m3c_cells,
#                    f'../L2/ALL/{integration_group}/m3c_cells.txt',
#                    downsample=1000000,
#                    random_state=0)
# 
#     
# cwd = pathlib.Path().absolute()
# with open('Snakefile') as f, open('../L2/ALL/Snakefile', 'w') as out_f:
#     snakefile_tmp = f.read()
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_MC_TYPE', str('CHN'))
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_TEMPLATE_DIR', str(cwd))
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_DATASET', 'CEMBA_3C')
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_CLUSTER_COL', str('L2'))
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_GROUP_KEY',
#                                           str('L3'))
#     snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_PLOT_KEY',
#                                           str('L2'))
#     snakefile_tmp = snakefile_tmp.replace(
#         'REPLACE_CATEGORICAL_KEY', str(['L2', 'L3', 'DissectionRegion']))
#     out_f.write(snakefile_tmp)

In [ ]:
all_mc_cells = mc_annot.get_index('cell')
all_m3c_cells = m3c_annot.get_index('cell')

In [ ]:
mc_cells = []
m3c_cells = []
for group_dir in pathlib.Path('../L2/ALL/').glob('InteGroup*'):
    mc_groups = pd.read_csv(f'{group_dir}/mc_integration_group.csv.gz',
                        index_col=0).squeeze()
    mc_cells += mc_groups.index.to_list()
    
    m3c_groups = pd.read_csv(f'{group_dir}/m3c_integration_group.csv.gz',
                        index_col=0).squeeze()
    m3c_cells += m3c_groups.index.to_list()

In [ ]:
l2_skipped_mc_cells = all_mc_cells[~all_mc_cells.isin(mc_cells)]

with open('l2_integration_skipped_mc_cells.txt', 'w') as f:
    f.write('\n'.join(l2_skipped_mc_cells))

In [ ]:
l2_skipped_m3c_cells = all_m3c_cells[~all_m3c_cells.isin(m3c_cells)]

with open('l2_integration_skipped_m3c_cells.txt', 'w') as f:
    f.write('\n'.join(l2_skipped_m3c_cells))

In [ ]:
l2_skipped_mc_cells.size

In [ ]:
l2_skipped_m3c_cells.size

### L4

In [ ]:
skipped_mc_cells = []
skipped_m3c_cells = []

for group_dir in pathlib.Path('../L2/ALL/').glob('InteGroup*'):
    mc_groups = pd.read_csv(f'{group_dir}/mc_integration_group.csv.gz',
                        index_col=0).squeeze()
    m3c_groups = pd.read_csv(f'{group_dir}/m3c_integration_group.csv.gz',
                         index_col=0).squeeze()
    
    all_groups = pd.concat([mc_groups, m3c_groups]).unique()
    
    for integration_group in all_groups:
        mc_cells = mc_groups[mc_groups == integration_group].index
        m3c_cells = m3c_groups[m3c_groups == integration_group].index
        integration_group = f'{group_dir.name}_{integration_group}'
        
        skip = False
        if mc_cells.size <= 150 or m3c_cells.size <= 150:
            print(integration_group, 'skipped due to small cell number')
            skip = True
        
        n_mc_l4 = mc_annot.sel(cell=mc_cells)['L4'].to_pandas().unique().size
        n_m3c_l4 = m3c_annot.sel(cell=m3c_cells)['L4'].to_pandas().unique().size
        if n_mc_l4 < 2 or n_m3c_l4 < 2:
            print(integration_group, 'skipped due to single L4 cluster')
            skip = True
        
        if skip:
            skipped_mc_cells += mc_cells.tolist()
            skipped_m3c_cells += m3c_cells.tolist()
            continue
        
        out_dir = pathlib.Path(f'../L4/ALL/{integration_group}')
        out_dir.mkdir(exist_ok=True, parents=True)

        save_index(mc_cells,
                   f'../L4/ALL/{integration_group}/mc_cells.txt',
                   downsample=1000000,
                   random_state=0)
        save_index(m3c_cells,
                   f'../L4/ALL/{integration_group}/m3c_cells.txt',
                   downsample=1000000,
                   random_state=0)

In [ ]:
cwd = pathlib.Path().absolute()
with open('Snakefile') as f, open('../L4/ALL/Snakefile', 'w') as out_f:
    snakefile_tmp = f.read()
    snakefile_tmp = snakefile_tmp.replace('REPLACE_MC_TYPE', str('CHN'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_TEMPLATE_DIR', str(cwd))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_DATASET', 'CEMBA_3C')
    snakefile_tmp = snakefile_tmp.replace('REPLACE_CLUSTER_COL', str('L3'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_GROUP_KEY',
                                          str('L4'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_PLOT_KEY',
                                          str('L4'))
    snakefile_tmp = snakefile_tmp.replace(
        'REPLACE_CATEGORICAL_KEY', str(['L3', 'L4', 'DissectionRegion']))
    out_f.write(snakefile_tmp)

### Check L4 cells correctness

In [ ]:
l4_dir = pathlib.Path('/home/qz/project/cemba/study/integration/mC_CEMBA_3C_Chrom100k/L4/ALL/')

# check to make sure L4 integroup is mutually exclusive
total_mc_cells = []
for p in l4_dir.glob('InteGroup*/mc_cells.txt'):
    cells = pd.read_csv(p, header=None, index_col=0).index.tolist()
    total_mc_cells += cells
assert pd.Series(total_mc_cells).duplicated().sum() == 0

total_m3c_cells = []
for p in l4_dir.glob('InteGroup*/m3c_cells.txt'):
    cells = pd.read_csv(p, header=None, index_col=0).index.tolist()
    total_m3c_cells += cells
assert pd.Series(total_m3c_cells).duplicated().sum() == 0

In [ ]:
# skipped cells are not in any L4 integration groups
assert pd.Series(skipped_mc_cells).isin(total_mc_cells).sum() == 0
assert pd.Series(skipped_m3c_cells).isin(total_m3c_cells).sum() == 0

In [ ]:
# L4 inte group cells and skipped cells together equal to all cells in the annot
cur_cells = pd.Index(skipped_mc_cells + total_mc_cells + l2_skipped_mc_cells.tolist())
all_mc_cells = mc_annot.get_index('cell')
assert cur_cells.isin(all_mc_cells).sum() == cur_cells.size
assert all_mc_cells.isin(cur_cells).sum() == all_mc_cells.size

cur_cells = pd.Index(skipped_m3c_cells + total_m3c_cells + l2_skipped_m3c_cells.tolist())
all_m3c_cells = m3c_annot.get_index('cell')
assert cur_cells.isin(all_m3c_cells).sum() == cur_cells.size
assert all_m3c_cells.isin(cur_cells).sum() == all_m3c_cells.size